In [ ]:
from pathlib import Path
import matplotlib as mpl

import matplotlib.pyplot as plt
import colorcet  # noqa: F401
from flow_analysis_comps.video_manipulation.control_class import videoControl
from flow_analysis_comps.Classic.extract_velocity import kymoAnalyser, videoDeltas
from flow_analysis_comps.Fourier.OrientationSpaceManager import orientationSpaceManager

plt.close("all")

%matplotlib widget

%load_ext autoreload
%autoreload 2

In [ ]:
video_root_folder = Path(r'C:\Users\vstaalduine\AMOLF-SHIMIZU Dropbox\Simon van Staalduine\034')
info_file_address = Path(video_root_folder / 'videoInfo.txt')
video_operator = videoControl(video_root_folder, info_file_address)
mean_img = video_operator.mean_img
print(mean_img.shape)

In [ ]:
kymographs = video_operator.get_kymographs()

In [ ]:
import numpy as np


video_deltas = videoDeltas(
    delta_x=video_operator.space_pixel_size,
    delta_t=video_operator.time_pixel_size
)

OSFilter = orientationSpaceManager(
    .25,
    freq_width=.5,
    K=16,
    x_spacing=video_operator.space_pixel_size,
    y_spacing=video_operator.time_pixel_size,
)

new_order=  12

for key, kymo in kymographs.items():
    analyser = kymoAnalyser(kymo, video_deltas=video_deltas)
    analyser.plot_kymo_fields()
    fig, ax = analyser.plot_summary()
    fig.savefig(video_root_folder / f"{key}_summary.png")
    image_l, image_r = analyser.kymograph_decomposed_directions
    invert_im = video_operator.video_info.mode == "brightfield"    
    OSFilter.demo_image(
        image_l,
        video_operator.space_pixel_size,
        video_operator.time_pixel_size,
        order=new_order,
        thresh_method="triangle",
        invert=invert_im,
        histo_thresh=.4,
        speed_extent=10
    )    
    OSFilter.demo_image(
        image_r,
        video_operator.space_pixel_size,
        video_operator.time_pixel_size,
        order=new_order,
        thresh_method="triangle",
        invert=invert_im,
        histo_thresh=.5,
        speed_extent=10
    )
    